In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 218.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 591.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.4 MB/s eta 0:00:00


In [ ]:
import re
from pathlib import Path
import pdfplumber

In [ ]:
def extraer_texto(pdf_path, margen=10):
    textos=[]
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt=page.extract_text() or ""
            textos.append(txt)
    return "\n\n".join(textos)

def procesar_tabla(lineas_tabla):
    filas=[[c.strip() for c in l.split("\t")] for l in lineas_tabla if l.strip()]
    if not filas:
        return ""
    return "\n".join(" ".join(f) for f in filas)

def limpiar_texto(texto):
    lineas=texto.splitlines()
    parrafos=[]
    par_act=""
    tabla_block=[]
    def _flush_parrafo():
        nonlocal par_act
        if par_act.strip():
            parrafos.append(par_act.strip())
            par_act=""
    def _flush_tabla():
        nonlocal tabla_block
        if tabla_block:
            tab=procesar_tabla(tabla_block)
            if tab:
                parrafos.append(tab)
            tabla_block=[]
    for l in lineas:
        l=l.strip()
        if (not l
            or re.fullmatch(r"[.\-_,\s]{3,}",l)
            or re.search(r"\bPágina\s*\d+",l,re.IGNORECASE)
            or "continúa en la página siguiente" in l.lower()
            or re.match(r"^https?://",l)
            or re.match(r"^\d+\s*$",l)
        ):
            continue
        l=re.sub(r"\.{3,}\s*\d+\s*$","",l)
        if "\t" in l:
            _flush_parrafo()
            tabla_block.append(l)
            continue
        else:
            _flush_tabla()
        if par_act and l and l[0].isupper() and par_act.rstrip()[-1] not in ".:?!»":
            _flush_parrafo()
        par_act=f"{par_act} {l}".strip() if par_act else l
        if l.endswith((".",":","?","!","»")):
            _flush_parrafo()
    _flush_tabla()
    _flush_parrafo()
    return "\n\n".join(parrafos)

def eliminar_dobles_espacios(texto):
    return re.sub(r" {2,}"," ",texto)

def pdf_a_txt_limpio(pdf_path, txt_path):
    raw=extraer_texto(pdf_path)
    clean=limpiar_texto(raw)
    final=eliminar_dobles_espacios(clean)
    Path(txt_path).write_text(final,encoding="utf-8")

PDF_ENTRADA="gsk-214094-plain-language-summary-part 1-eses.pdf"
TXT_SALIDA ="gsk-214094-plain-language-summary-part 1-eses_LIMPIO.txt"
pdf_a_txt_limpio(PDF_ENTRADA,TXT_SALIDA)